In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sec_edgar.data.preprocessing.preprocess_4form_files import pre_process_4form_archive_files
from sec_edgar.data.preprocessing.preprocess_master_idx import pre_process_master_idx_content
from sec_edgar.data.processing.process_4form_files import Process4FormFiles
from sec_edgar.data.processing.process_append_historical_prices import ProcessAppendHistoricalPrices
from sec_edgar.data.utils.cik_mapping_util import CikMappingUtil
from sec_edgar.stock_historical_data.av_historical_data_request import AVHistoricalDataRequest
import pandas as pd

from sec_edgar.utils.evaluation_metrics import PerformanceMetric01, PerformanceMetric02, PerformanceMetric03
from sec_edgar.utils.performance_evaluations import PerformanceEvaluations4Form, PerformanceEvaluations4FormYear

In [2]:
pd.set_option('display.width', 2500)      # max table width to display

In [3]:
## Loading of data set

In [4]:
data_set_name = "us_market_allderv_AandB_1019.py"
base_path = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))

path_processed_datasets = base_path + '/Data/processed_datasets/'
file_name = path_processed_datasets + data_set_name + ".csv"

processed_4form_df = pd.read_csv(file_name, index_col=0)

In [5]:
print(file_name)
display(processed_4form_df)
display(processed_4form_df.columns.to_list())

C:\Users\andre\OneDrive\Documents\MathModels\sec_edgar/Data/processed_datasets/us_market_allderv_AandB_1019.py.csv


,CIK,Company_Names,Date_Filed,index_id,index_id_min,n_transactions,shares_percent_changes_mean,transactionSharesAdjust,date_ft_delta_mean,Filename,...,Shifted Price pct_change (10)(10 days 00:00:00),Shifted Price pct_change (10)(-10 days +00:00:00),Shifted Price pct_change (15)(15 days 00:00:00),Shifted Price pct_change (15)(-15 days +00:00:00),Shifted Price pct_change (21)(21 days 00:00:00),Shifted Price pct_change (21)(-21 days +00:00:00),Shifted Price pct_change (31)(31 days 00:00:00),Shifted Price pct_change (31)(-31 days +00:00:00),Shifted Price pct_change (42)(42 days 00:00:00),Shifted Price pct_change (42)(-42 days +00:00:00)
index,,,,,,,,,,,,,,,,,,,,,
48441,27904,DELTA AIR LINES INC /DE/,2010-01-04,[140546],140546,1,NaN,0.00,0 days 00:00:00,['edgar/data/27904/0001019687-10-000008.txt'],...,-0.071374,0.086661,-0.049117,0.075472,-0.043511,0.187556,0.140489,0.352113,0.174694,0.701164
107523,935703,DOLLAR TREE INC,2010-01-04,[578414 578415 578416 578417 578418],578414,5,0.250942,3544.61,3 days 00:00:00,['edgar/data/935703/0001127602-10-000234.txt'\...,...,0.005482,0.000612,-0.034031,0.014925,0.143033,0.017983,0.211652,0.053858,0.232375,-0.000204
48728,28412,COMERICA INC /NEW/,2010-01-04,[79905 79906 79907 79908 79909 79910 79911 799...,79905,13,0.018748,1152.00,5 days 00:00:00,['edgar/data/28412/0000028412-10-000001.txt'\n...,...,0.011134,0.068302,0.030569,0.152212,0.014648,0.167237,0.143071,0.266620,0.203143,0.213538
75418,72741,NORTHEAST UTILITIES,2010-01-04,[378340 378341 378342 378343],378340,4,0.146730,9435.00,0 days 00:00:00,['edgar/data/72741/0000072741-10-000004.txt'\n...,...,-0.040546,-0.000762,-0.051874,0.003069,0.028191,0.022868,0.074148,0.055285,0.109829,0.049897
97444,882835,ROPER INDUSTRIES INC,2010-01-04,[229120 229121 229122 229123],229120,4,-0.027286,0.00,5 days 00:00:00,['edgar/data/882835/0001209191-10-000310.txt'],...,-0.046095,-0.010336,-0.029718,-0.028972,0.074132,-0.028318,0.083082,-0.027359,0.165974,-0.048330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61641,49071,HUMANA INC,2019-12-31,[795842 795844 795845 795846 795847 795848 795...,795842,14,NaN,0.00,0 days 00:00:00,['edgar/data/49071/0000049071-19-000146.txt'\n...,...,-0.047265,-0.005206,-0.013514,0.024926,0.016773,0.056768,-0.189329,0.066934,-0.024946,0.088031
3960,1018724,AMAZON COM INC,2019-12-31,[334103],334103,1,NaN,0.00,4 days 00:00:00,['edgar/data/1018724/0000899243-19-030277.txt'],...,-0.029111,0.067372,0.101078,0.069387,0.138002,0.082767,-0.100004,0.033996,0.008402,0.076832
65091,54480,KANSAS CITY SOUTHERN,2019-12-31,[827929 827930 827932 827933 827934 827935 827...,827929,10,-0.001570,-285.00,11 days 00:00:00,['edgar/data/54480/0000054480-19-000247.txt'],...,0.036840,0.001612,0.076889,0.037715,0.063520,0.046904,-0.281522,0.047828,-0.197721,0.069084


['CIK',
 'Company_Names',
 'Date_Filed',
 'index_id',
 'index_id_min',
 'n_transactions',
 'shares_percent_changes_mean',
 'transactionSharesAdjust',
 'date_ft_delta_mean',
 'Filename',
 'Form_Type',
 'conversionOrExercisePrice',
 'date_ft_delta',
 'deemedExecutionDate',
 'directOrIndirectOwnership',
 'doc_name',
 'exerciseDate',
 'expirationDate',
 'file_url',
 'isDirector',
 'isOfficer',
 'isOther',
 'isTenPercentOwner',
 'issuerName',
 'issuerTradingSymbol',
 'officerTitle',
 'otherText',
 'periodOfReport',
 'quarter',
 'rptOwnerName',
 'securityTitle',
 'sharesOwnedFollowingTransaction',
 'shares_percent_changes',
 'signatureDate',
 'transactionAcquiredDisposedCode',
 'transactionCode',
 'transactionDate',
 'transactionFormType',
 'transactionPricePerShare',
 'transactionShares',
 'transactionTotalValue',
 'transaction_table',
 'transaction_type',
 'underlyingSecurityShares',
 'underlyingSecurityTitle',
 'underlyingSecurityValue',
 'valueOwnedFollowingTransaction',
 'year',
 'my_de

## Performance Metrics Evaluations

In [6]:
pe_4form = PerformanceEvaluations4Form(processed_4form_df, n_sub_set=5)

In [7]:
days = 42
column_label_eval = 'Price pct_change ({days})'.format(days=days)
shifted_columns_label_eval = 'Shifted Price pct_change ({days})(-{days} days +00:00:00)'.format(days=days)

In [8]:

# PerformanceMetric01 evaluation:
metrics_df_01, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric01,
                                        pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation: ")
display(metrics_df_01)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_02b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric02,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation: ")
display(metrics_df_02)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                        pm_kwargs=dict(column_label_eval=column_label_eval,
                                                       shifted_columns_label_eval=
                                                       shifted_columns_label_eval))

metrics_df_03b, _ = pe_4form.eval_metric(performance_metric_ref=PerformanceMetric03,
                                         pm_kwargs=dict(column_label_eval=column_label_eval,
                                                        shifted_columns_label_eval=
                                                        'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation: ")
display(metrics_df_03)
print("-" * 75)


PerformanceMetric01 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.02392,0.025572,0.022357,0.025454,0.022292,0.023926
benchmark,0.02113,0.023392,0.020648,0.022177,0.019059,0.020375
is_greater_prob,0.590588,0.6,0.529412,0.635294,0.6,0.588235
is_pos_tse_ok,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,True,True,True,True
Positive_tsa_std,0.023454,0.018075,0.016464,0.021724,0.018922,0.036587
benchmark_std,0.017972,0.015029,0.015719,0.013612,0.013191,0.028046
statistic,1.946492,0.854766,0.692141,1.178711,1.292105,0.710127
pvalue,0.025974,0.196969,0.244903,0.120248,0.099154,0.239338


---------------------------------------------------------------------------

PerformanceMetric02 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.02392,0.025572,0.022357,0.025454,0.022292,0.023926
benchmark,0.020797,0.024622,0.013626,0.026161,0.019198,0.020376
is_greater_prob,0.562353,0.505882,0.658824,0.529412,0.529412,0.588235
is_pos_tse_ok,True,True,True,False,True,True
is_greater_prob_ok,True,True,True,True,True,True
is_performance_ok,True,True,True,False,True,True
Positive_tsa_std,0.023454,0.018075,0.016464,0.021724,0.018922,0.036587
benchmark_std,0.029494,0.030562,0.021353,0.025487,0.024109,0.040883
statistic,1.708818,0.246665,2.98545,-0.194655,0.930604,0.596554
pvalue,0.043935,0.402769,0.001642,0.577048,0.176734,0.275809


---------------------------------------------------------------------------

PerformanceMetric03 evaluation: 


,ss_all,ss_0,ss_1,ss_2,ss_3,ss_4
Positive_tsa,0.022887,0.024425,0.021194,0.023595,0.022205,0.022672
benchmark,0.019835,0.022653,0.015324,0.023093,0.019461,0.017774
is_greater_prob,0.502074,0.501722,0.506653,0.495301,0.498124,0.510065
is_pos_tse_ok,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,False,False,True
is_performance_ok,True,True,True,False,False,True
Positive_tsa_std,0.109742,0.11262,0.108545,0.112589,0.103151,0.10995
benchmark_std,0.130322,0.165179,0.113414,0.119838,0.107363,0.125424
statistic,2.653241,0.658138,2.653265,0.206572,1.12624,1.629716
pvalue,0.003988,0.255233,0.003992,0.418174,0.13005,0.051607


---------------------------------------------------------------------------


In [9]:
print("=" * 90)
pe_4form_year = PerformanceEvaluations4FormYear(processed_4form_df)

# PerformanceMetric01 evaluation:
metrics_df_01_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric01,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval))
print("\nPerformanceMetric01 evaluation (Year): ")
display(metrics_df_01_year)
print("-" * 75)

# PerformanceMetric02 evaluation:
metrics_df_02_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_02b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric02,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))
print("\nPerformanceMetric02 evaluation (Year): ")
display(metrics_df_02_year)
print("-" * 75)

# PerformanceMetric03 evaluation:
metrics_df_03_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                  pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                 shifted_columns_label_eval=
                                                                 shifted_columns_label_eval))

metrics_df_03b_year, _ = pe_4form_year.eval_metric(performance_metric_ref=PerformanceMetric03,
                                                   pm_kwargs=dict(column_label_eval=column_label_eval,
                                                                  shifted_columns_label_eval=
                                                                  'Shifted Price pct_change (10)(10 days 00:00:00)'))

print("\nPerformanceMetric03 evaluation (Year): ")
display(metrics_df_03_year)
print("-" * 75)

print("=" * 90)


PerformanceMetric01 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.02392,0.04779,0.01216,0.023083,0.041262,0.02067,-0.014417,0.053008,0.025921,0.004898,0.031754
benchmark,0.02113,0.042036,0.009983,0.025869,0.0343,0.018085,-0.01998,0.038738,0.022375,-0.004231,0.022967
is_greater_prob,0.590588,0.576142,0.506266,0.476071,0.578554,0.566085,0.564792,0.631068,0.53125,0.630695,0.613365
is_pos_tse_ok,True,True,True,False,True,True,True,True,True,True,True
is_greater_prob_ok,True,True,True,False,True,True,True,True,True,True,True
is_performance_ok,True,True,True,False,True,True,True,True,True,True,True
Positive_tsa_std,0.023454,0.070329,0.072182,0.070366,0.064828,0.064935,0.06611,0.068176,0.065579,0.062692,0.056628
benchmark_std,0.017972,0.047975,0.056036,0.045313,0.048588,0.035499,0.045234,0.058042,0.052487,0.044276,0.044735
statistic,1.946492,1.341619,0.475747,-0.66343,1.720856,0.699655,1.404586,3.234856,0.861028,2.429034,2.49241
pvalue,0.025974,0.090079,0.317197,0.746359,0.042847,0.242203,0.080287,0.000633,0.194742,0.007687,0.006445


---------------------------------------------------------------------------

PerformanceMetric02 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.02392,0.04779,0.01216,0.023083,0.041262,0.02067,-0.014417,0.053008,0.025921,0.004898,0.031754
benchmark,0.020797,0.030758,0.013929,0.038809,0.047765,0.026142,0.001414,0.024428,0.030373,-0.010126,0.036173
is_greater_prob,0.562353,0.611675,0.521303,0.382872,0.401496,0.466334,0.415648,0.648058,0.4375,0.601918,0.470167
is_pos_tse_ok,True,True,False,False,False,False,False,True,False,True,False
is_greater_prob_ok,True,True,True,False,False,False,False,True,False,True,False
is_performance_ok,True,True,False,False,False,False,False,True,False,True,False
Positive_tsa_std,0.023454,0.070329,0.072182,0.070366,0.064828,0.064935,0.06611,0.068176,0.065579,0.062692,0.056628
benchmark_std,0.029494,0.076144,0.114515,0.074922,0.061346,0.076698,0.064145,0.070434,0.071949,0.068704,0.068162
statistic,1.708818,3.261594,-0.261067,-3.048496,-1.45899,-1.090334,-3.475728,5.917977,-0.932815,3.298745,-1.020843
pvalue,0.043935,0.000578,0.60294,0.998812,0.927519,0.862048,0.999732,0.0,0.824406,0.000506,0.846183


---------------------------------------------------------------------------

PerformanceMetric03 evaluation (Year): 


,ss_all,ss_2010,ss_2011,ss_2012,ss_2013,ss_2014,ss_2015,ss_2016,ss_2017,ss_2018,ss_2019
Positive_tsa,0.022887,0.0439,0.011634,0.020474,0.039468,0.024306,-0.01442,0.050484,0.02343,0.002847,0.027166
benchmark,0.019835,0.032947,0.006714,0.032634,0.044135,0.02851,-0.004856,0.020861,0.024578,-0.011182,0.026852
is_greater_prob,0.502074,0.549767,0.490035,0.461834,0.482091,0.476304,0.46078,0.558368,0.48026,0.556902,0.494363
is_pos_tse_ok,True,True,True,False,False,False,False,True,False,True,True
is_greater_prob_ok,True,True,False,False,False,False,False,True,False,True,False
is_performance_ok,True,True,False,False,False,False,False,True,False,True,False
Positive_tsa_std,0.109742,0.122269,0.141398,0.115411,0.090631,0.083148,0.104375,0.114125,0.093016,0.100693,0.100326
benchmark_std,0.130322,0.12558,0.233187,0.124817,0.095515,0.093272,0.102996,0.121857,0.103779,0.110694,0.11132
statistic,2.653241,3.036282,0.866638,-3.264565,-1.611147,-1.537514,-3.009455,8.194445,-0.382336,4.506792,0.100609
pvalue,0.003988,0.001204,0.193098,0.999447,0.946388,0.937878,0.998684,0.0,0.648885,0.000003,0.459932


---------------------------------------------------------------------------


## OutLayers Rem Evaluation test

In [10]:
post_processed_4form_df = processed_4form_df.copy()
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df.my_derivative_types.isin(['AB', 'B'])]
post_processed_4form_df = post_processed_4form_df[post_processed_4form_df['transactionSharesAdjust'] > 0]

In [11]:
def _ecdf(sample):
    sample = np.atleast_1d(sample)
    quantiles, counts = np.unique(sample, return_counts=True)
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob


def _get_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[np.logical_and(cumprob > 0.5 * ci, cumprob < 1. - 0.5 * ci)]

def _get_left_sub_set_in_ci(quantiles, cumprob, ci=10 / 100):
    return quantiles[cumprob < 1. - ci]

quantiles, cumprob = _ecdf(post_processed_4form_df[column_label_eval].values)
results_dict = dict()
results_dict["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()
results_dict["E[R|0.25<P(R)<0.75]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.25 * 2).mean()

quantiles, cumprob = _ecdf(post_processed_4form_df[shifted_columns_label_eval].values)
results_dict_bench = dict()
results_dict_bench["E[R|0.05<P(R)<0.95]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.05 * 2).mean()
results_dict_bench["E[R|0.10<P(R)<0.90]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.1 * 2).mean()
results_dict_bench["E[R|0.15<P(R)<0.85]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.15 * 2).mean()
results_dict_bench["E[R|0.20<P(R)<0.80]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.2 * 2).mean()
results_dict_bench["E[R|0.25<P(R)<0.75]"] = _get_sub_set_in_ci(quantiles, cumprob, ci=0.25 * 2).mean()

In [12]:
print("\n MostProb Scenarios results: \n")
display(pd.Series(results_dict))
print("\n MostProb Scenarios results Bench: \n")
display(pd.Series(results_dict_bench))


 MostProb Scenarios results: 



E[R|0.05<P(R)<0.95]    0.023344
E[R|0.10<P(R)<0.90]    0.023538
E[R|0.15<P(R)<0.85]    0.023698
E[R|0.20<P(R)<0.80]    0.023816
E[R|0.25<P(R)<0.75]    0.023914
dtype: float64


 MostProb Scenarios results Bench: 



E[R|0.05<P(R)<0.95]    0.019364
E[R|0.10<P(R)<0.90]    0.019667
E[R|0.15<P(R)<0.85]    0.019977
E[R|0.20<P(R)<0.80]    0.020183
E[R|0.25<P(R)<0.75]    0.020432
dtype: float64